<a href="https://colab.research.google.com/github/shiba01Git/Youtube_chatdat_Python/blob/main/youtubechat_%E6%9C%AA%E5%AE%8C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install matplotlib

In [ ]:
pip install yt-dlp pytchat

In [ ]:
pip install japanize-matplotlib

In [ ]:
import sys
from yt_dlp import YoutubeDL
import pytchat
from matplotlib import pyplot as plt


# チャットデータの取得
def chat_get(youtube_url):
        video_id = extract_video_id(youtube_url)
        livechat = pytchat.create(video_id=video_id)
        while livechat.is_alive():
            chatdata = livechat.get()
            if(len(chatdata.items) > 0):
                for c in chatdata.items:
                    print(f"{c.datetime} {c.elapsedTime} {c.author.name} {c.message} {c.amountString}")
                    txt_in.append(c.elapsedTime)
                # .txtで保存
                with open(video_id + '_chatdata.txt', 'w', encoding='utf-8', newline='\n') as f:
                    f.writelines(txt_in)

            else:
                break

# URLからvideo_id抜き出し
def extract_video_id(url):
    with YoutubeDL() as ydl:
        info_dict = ydl.extract_info(url, download=False)
        return info_dict.get("id", None)

#コマンドから引数取得
#if len(sys.argv) != 2:
    #print("Usage: python script.py ")
    #sys.exit(1)

#実行
txt_in = []
#youtube_url = sys.argv[1]
#youtube_url = "https://www.youtube.com/watch?v=EZ_S2eGmbjU&t=452s"
youtube_url = "https://www.youtube.com/live/p51d135SeIs?si=meQT86sdAw5OaRi9"
txt_in = []
chat_get(youtube_url)

# h:m:s 表記のリスト（例: "45:15" の場合は h が省略されている）
#txt_in = ["1:30:45", "45:15", "2::30", "10"]




In [ ]:
import japanize_matplotlib

# 秒数に変換する関数
def hms_to_seconds(hms):
    parts = hms.split(':')
    #print(parts)
    #それぞれの要素が空の場合はデフォルトで0に設定
    h = int(parts[0]) if len(parts) > 2 and parts[0] else 0
    m = int(parts[-2]) if len(parts) > 1 and parts[-2] else 0
    s = int(parts[-1]) if parts[-1] else 0
    return h * 3600 + m * 60 + s

# txt_inの各要素を秒数に変換
seconds_list = [hms_to_seconds(time) for time in txt_in]

#print(seconds_list)

# 10秒ごとの範囲でカウントする関数

def count_elements_in_ranges(lst, interval):
    max_value = max(lst)

    # 結果を格納するリスト
    playback_time_list = []   # 範囲の文字列を格納するリスト
    count_list = [] # カウント数を格納するリスト

    # 0〜10秒, 10〜20秒, ... の範囲ごとにカウント
    for start in range(0, max_value + interval, interval):
        end = start + interval
        count = sum(start <= x < end for x in lst)  # 指定した範囲に含まれる要素をカウント

        # 範囲の文字列を作成してplayback_time_listに追加
        #playback_time_list.append(f"{start}秒以上{end}秒未満")
        playback_time_list.append(end)
        # カウント数をcount_listに追加
        count_list.append(count)

    return playback_time_list,count_list


# 実行
interval = 60
playback_time_list, count_list = count_elements_in_ranges(seconds_list, interval)


#x = [s for s in range(0,max(playback_time_list),interval)]
max_time = max(playback_time_list)

#単位を分に直す
#max_time_minutes = max_time / 60
#interval_minutes = 20 / 60

x = [s for s in range(0,max_time,interval)]
y = count_list

# フォントを全て読み込み
#fonts = set([f.name for f in matplotlib.font_manager.fontManager.ttflist])


fig, ax = plt.subplots(figsize=(10,4), dpi=600 )
xticks_array = [s for s in range(0,max_time,600)]

ax.plot(x,y)
ax.set_xlabel("動画時間",size = "small")
ax.set_ylabel("コメント数",size = "small")
plt.xticks(xticks_array, fontsize=7, rotation=45)
ax.minorticks_on()
plt.grid(color = "gray", linestyle="--")
ax.minorticks_on()
plt.tight_layout()
plt.show()